In [1]:
#import xgboost as xgb
#from sklearn.metrics import mean_squared_error

In [2]:
import pandas as pd
import numpy as np

In [3]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [4]:
train.head(3)

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A
0,27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,Q2,2016,NaT,10,EUR,5272800.0,EUR,5272800.0,Closed Lost,Prod_Category_A_None
1,27760,EMEA,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,Q1,2016,NaT,0,EUR,48230.0,EUR,48230.0,Closed Won,Prod_Category_A_None
2,27446,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,...,Q1,2016,NaT,0,USD,83865.6,USD,83865.6,Closed Won,Prod_Category_A_None


In [5]:
train=train.loc[(train.Stage=="Closed Lost") | (train.Stage=="Closed Won")]

In [6]:
train.Stage=train.Stage.replace(['Closed Won','Closed Lost'],[1,0])

In [7]:
train.head(2)

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A
0,27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,Q2,2016,NaT,10,EUR,5272800.0,EUR,5272800.0,0,Prod_Category_A_None
1,27760,EMEA,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,Q1,2016,NaT,0,EUR,48230.0,EUR,48230.0,1,Prod_Category_A_None


Removiendo Columnas que son Incesarias para el analisis:
    -No cuentan con informacion
    -No aportan al analisis

In [8]:
train=train.drop(columns=["Submitted_for_Approval","Prod_Category_A","Last_Activity","Actual_Delivery_Date"])

Separo la variable a predecir:

In [9]:
stage=train.Stage

In [10]:
stage.value_counts()

1    9533
0    7350
Name: Stage, dtype: int64

In [11]:
train.shape

(16883, 48)

Para este Primer analisis me quedare con las 8 primeras Variables del Set de datos:

In [12]:
train_1=train.iloc[:,:8]

In [13]:
train_1.head(2)

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code
0,27761,EMEA,None,1,1,1,1,Bureaucratic_Code_4
1,27760,EMEA,None,0,0,0,0,Bureaucratic_Code_4


In [14]:
train_1['Pricing, Delivery_Terms_Quote_Appr'].unique(),train['Pricing, Delivery_Terms_Quote_Appr'].shape

(array([1, 0]), (16883,))

In [15]:
train_1['Pricing, Delivery_Terms_Approved'].unique(),train['Pricing, Delivery_Terms_Approved'].shape

(array([1, 0]), (16883,))

In [16]:
train_1['Bureaucratic_Code_0_Approval'].unique(),train['Bureaucratic_Code_0_Approval'].shape

(array([1, 0]), (16883,))

In [17]:
train_1['Bureaucratic_Code_0_Approved'].unique(),train['Bureaucratic_Code_0_Approved'].shape

(array([1, 0]), (16883,))

Usare One Hot Encoding sobre los codigos Burocraticos:

In [18]:
#from sklearn.preprocessing import LabelBinarizer

In [19]:
#y = LabelBinarizer().fit_transform(train_1.Bureaucratic_Code)

In [20]:
#Bureaucratic_Code_OHE=pd.DataFrame(y).rename(columns={0:"Bureaucratic_Code_0",1:"Bureaucratic_Code_1",2:"Bureaucratic_Code_2",3:"Bureaucratic_Code_3",4:"Bureaucratic_Code_4",5:"Bureaucratic_Code_5",6:"Bureaucratic_Code_6"})

In [21]:
y = pd.get_dummies(train_1.Bureaucratic_Code, prefix='OHE')

In [22]:
y.head(2)

,OHE_Bureaucratic_Code_0,OHE_Bureaucratic_Code_1,OHE_Bureaucratic_Code_2,OHE_Bureaucratic_Code_3,OHE_Bureaucratic_Code_4,OHE_Bureaucratic_Code_5,OHE_Bureaucratic_Code_6
0,0,0,0,0,1,0,0
1,0,0,0,0,1,0,0


In [23]:
train_1["OHE_Bureaucratic_Code_0"]=y["OHE_Bureaucratic_Code_0"]
train_1["OHE_Bureaucratic_Code_1"]=y["OHE_Bureaucratic_Code_1"]
train_1["OHE_Bureaucratic_Code_2"]=y["OHE_Bureaucratic_Code_2"]
train_1["OHE_Bureaucratic_Code_3"]=y["OHE_Bureaucratic_Code_3"]
train_1["OHE_Bureaucratic_Code_4"]=y["OHE_Bureaucratic_Code_4"]
train_1["OHE_Bureaucratic_Code_5"]=y["OHE_Bureaucratic_Code_5"]
train_1["OHE_Bureaucratic_Code_6"]=y["OHE_Bureaucratic_Code_6"]

In [24]:
train_1=train_1.drop(columns=["Region","Territory","Bureaucratic_Code","ID"])

In [25]:
train_1.head(2)

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,OHE_Bureaucratic_Code_0,OHE_Bureaucratic_Code_1,OHE_Bureaucratic_Code_2,OHE_Bureaucratic_Code_3,OHE_Bureaucratic_Code_4,OHE_Bureaucratic_Code_5,OHE_Bureaucratic_Code_6
0,1,1,1,1,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,1,0,0


In [26]:
from sklearn.model_selection import train_test_split
import xgboost

In [27]:
#train_1 = xgb.DMatrix(data=train_1,label=stage)

In [28]:
X_train, X_test, y_train, y_test=train_test_split(train_1,stage,test_size=0.5,random_state=1)

In [29]:
#y_train=y_train.to_frame()
#y_train=y_train.Stage.values.reshape(len(y_train), 1)
#y_train=y_train.reshape(-1,1)

In [30]:
X_train.shape,X_test.shape,y_train.shape

((8441, 11), (8442, 11), (8441,))

In [31]:
X_valid, X_test, y_valid, y_test=train_test_split(X_test,y_test,test_size=0.5,random_state=123)

In [32]:
#y_valid=y_valid.to_frame()
#y_test=y_test.to_frame()

In [33]:
#X_valid.shape,X_test.shape,y_valid.shape,y_test.shape

In [34]:
xgb=xgboost.XGBClassifier()

In [35]:
parameters={
    'nThreads':[1],
    'objective':['binary:logistic'],
    'learning rate':[0.01,0.05,1],
    'n_estimators':[100,200],
}

In [36]:
from sklearn.model_selection import GridSearchCV

In [37]:
fit_params={'early_stopping_rounds':10,
           'eval_metric':'logloss',
           'eval_set':[(X_test,y_test)]}

In [38]:
fit_params2={'early_stopping_rounds':10,
           'eval_metric':'auc',
           'eval_set':[(X_test,y_test)]}

In [39]:
clf=GridSearchCV(xgb,parameters,cv=3,scoring='accuracy')

In [40]:
X_train.shape,y_train.shape

((8441, 11), (8441,))

In [71]:
#clf.fit(X_train, y_train.ravel(), **fit_params)

In [42]:
clf.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning rate=0.01, learning_rate=0.300000012, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', nThreads=1, n_estimators=100, n_jobs=4,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [43]:
clf.best_score_

0.6720779661019805

In [44]:
from sklearn.metrics import accuracy_score

In [45]:
best_xgboost=clf.best_estimator_

In [46]:
best_xgboost.predict(X_valid)

array([1, 1, 1, ..., 1, 1, 0])

In [47]:
y_pred=best_xgboost.predict(X_valid)

In [48]:
comp=pd.DataFrame({'real':y_valid,'pred':y_pred})

In [49]:
comp.head(3)

,real,pred
12726,0,1
12844,0,1
11670,1,1


In [50]:
ac=accuracy_score(y_valid,y_pred)

In [51]:
ac

0.6806443970623075

OtherModelOfClassifation

In [52]:
import xgboost as xgb


In [53]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(X_train, y_train)

/home/rogger/.local/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:28:28] WARNING: /tmp/user/1000/pip-install-3m3911f1/xgboost/build/temp.linux-x86_64-3.7/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [54]:
y_pred = xgb_model.predict(X_valid)

In [55]:
comp=pd.DataFrame({'real':y_valid,'pred':y_pred})

In [56]:
ac=accuracy_score(y_valid,y_pred)

In [57]:
ac

0.6813551291163231

MODELO ERRONEO

In [58]:
from sklearn.model_selection import  KFold
from sklearn.metrics import mean_squared_error

In [58]:
train_1.head(3)

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,OHE_Bureaucratic_Code_0,OHE_Bureaucratic_Code_1,OHE_Bureaucratic_Code_2,OHE_Bureaucratic_Code_3,OHE_Bureaucratic_Code_4,OHE_Bureaucratic_Code_5,OHE_Bureaucratic_Code_6
0,1,1,1,1,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,1,0,0


In [60]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores = []

In [61]:
for train_index, test_index in kfold.split(train_1):   
    X_train, X_test = train_1.iloc[train_index], train_1.iloc[test_index]
    y_train, y_test = stage.iloc[train_index], stage.iloc[test_index]

    xgb_model = xgb.XGBRegressor(objective="reg:linear")
    xgb_model.fit(X_train, y_train)
    
    y_pred = xgb_model.predict(X_test)
    
    scores.append(mean_squared_error(y_test, y_pred))

[01:17:31] WARNING: /tmp/user/1000/pip-install-3m3911f1/xgboost/build/temp.linux-x86_64-3.7/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[01:17:32] WARNING: /tmp/user/1000/pip-install-3m3911f1/xgboost/build/temp.linux-x86_64-3.7/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[01:17:32] WARNING: /tmp/user/1000/pip-install-3m3911f1/xgboost/build/temp.linux-x86_64-3.7/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[01:17:32] WARNING: /tmp/user/1000/pip-install-3m3911f1/xgboost/build/temp.linux-x86_64-3.7/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[01:17:32] WARNING: /tmp/user/1000/pip-install-3m3911f1/xgboost/build/temp.linux-x86_64-3.7/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


In [62]:
comp=pd.DataFrame({'real':y_test,'pred':y_pred})

In [63]:
y_test.size

3376

In [64]:
y_pred.size

3376

In [65]:
(comp.pred>0.5)
comp[comp.pred>=0.5]=1

In [66]:
comp[comp.pred<0.5]=1

In [67]:
comp.head()

,real,pred
1,1,1.0
5,1,1.0
7,1,1.0
9,1,1.0
11,1,1.0


In [68]:
ac=accuracy_score(comp.real,comp.pred)

In [69]:
ac

1.0

In [ ]:
#def display_scores(scores):
 #   print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))

In [ ]:
#display_scores(np.sqrt(scores))

Regressor

In [89]:
X_train, X_test, y_train, y_test = \
    train_test_split(train_1, stage, test_size=0.2, random_state=123)

In [90]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [91]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=10, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [92]:
preds = xg_reg.predict(X_test)

In [93]:
comp=pd.DataFrame({'real':y_test,'pred':preds})

In [94]:
comp[comp.pred>=0.5]=1
comp[comp.pred<0.5]=0

In [95]:
comp

,real,pred
15610,1,1.0
1888,1,1.0
12232,1,1.0
854,1,1.0
8558,1,1.0
6872,0,0.0
4312,1,1.0
14667,1,1.0
15971,1,1.0
7188,1,1.0


In [96]:
ac=accuracy_score(comp.real,comp.pred)

In [97]:
ac

1.0